In [ ]:
addprocs(3)

using JOcTree
using MaxwellUtils
using MaxwellFrequency
using jInv.LinearSolvers
using jInv.InverseSolve
using MaxwellNatSource
using jInv.Utils
using jInv.Mesh

## Read mesh and true model

In [ ]:
Minv = importOcTreeMeshRoman("meshInv.txt")
sigmamodel = importOcTreeModelRoman("model_blocks.con", Minv);
;

## Setup survey

In [ ]:
datafile = [ "data_locations.txt",
             "trx_dummy.txt",
             "receiver_locations.txt",
             "frequencies.txt" ]

topofile = 0.

trxTmp, h, itopo = setupMeshParam(datafile, topofile, 
                               Minv.n,Minv.x0,Minv.n.*Minv.h;
                               only_loc=true)

frq = [10. 30. 100.]

trx = Array{TransmitterOmega}(length(frq))

for (i, f) in enumerate(frq)
    trx[i] = TransmitterOmega(copy(trxTmp[1].Srcs), 
                              2*pi*f, 
                              copy(trxTmp[1].Recs), 
                              copy(trxTmp[1].Dobs), 
                              copy(trxTmp[1].Wd))
end
;

## Generate initial model

In [ ]:
halfSpaceCond = 1e-2
backCond      = 1.e-8

gcc = getCellCenteredGrid(Minv)
isactive = gcc[:,3].<0.
nactive    = sum(isactive)
nnotactive = length(isactive) - nactive

# model a half space
sigma    = ones(nactive)*halfSpaceCond
sigmaBck = ones(nnotactive)*backCond

Iact    = speye(Bool,Minv.nc)
Iact    = Iact[:,find(isactive)]
IactBck = speye(Bool,Minv.nc)
IactBck = IactBck[:,find(!isactive)]
;

## Generate interpolation

In [ ]:
tic()
Obs = getAllObs( trx, Minv )
toc()
;

## Setup pFor

In [ ]:
linSolParam = getMUMPSsolver([],1,0,2)

nFreqs = length(trx)
pFor = Array(RemoteChannel,nFreqs)
workerList = workers()
nw = length(workerList)
for i = 1:nFreqs
   Sources = Array(Complex128, 0, 0)
   fields = Array(Complex128, 0, 0)
   pFor[i] = initRemoteChannel(getMaxwellFreqParam, workerList[i%nw+1],
                               Minv, Sources, Obs[i], fields,
                               trx[i].omega, linSolParam)
end


## Calculate Sources

In [ ]:
tic()
pFor = calcMTSources(sigmamodel, pFor, true)
toc()

## Calculate true data

In [ ]:
tic()
DD, pFor = getData( sigmamodel, pFor, ones(length(pFor)), true );
toc()

In [ ]:
println("Setup Inverse Param")

Dobs  = Array{Array{Complex128}}(nFreqs)
Wd    = Array{Array{Complex128}}(nFreqs)


for itx in 1:length(trx)
    trx[itx].Dobs = calcMTdata(fetch(DD[itx]))

    trx[itx].Wd = complex( 1.0 ./ (abs(real(trx[itx].Dobs))*0.01+1.e-5) ,
                           1.0 ./ (abs(imag(trx[itx].Dobs))*0.01+1.e-5) );

    Dobs[itx] = trx[itx].Dobs
      Wd[itx] = trx[itx].Wd
end
;

## Setup pInv

In [ ]:
# lower bounds
BL = 1e-6
# Higher bounds
BH = 1e+4
# misfit function
misfun = misRatio

# Regularization function
regfun = wdiffusionReg
# parameters for the regularization function
regparams = [sqrt(1.0), sqrt(1.0), sqrt(1.0), 5e-7]  # alphax  alphay  alphaz  alphas

#  inner CG iter
cgit = 10 
# maximum iter for the inversion
maxit = 6

# model parameter is log conductivity
modfun = expMod

beta = 1e-32
;

sigmaBackground = IactBck * sigmaBck
mref = fill(log(halfSpaceCond), size(Iact,2))

boundsLow  = fill(log(BL),size(Iact,2)) 
boundsHigh = fill(log(BH),size(Iact,2))    

pMisRF = getMisfitParam(pFor, Wd, Dobs, misfun,Iact,sigmaBackground)
;

regfunw(m,mreff,Mm) = wdiffusionReg(m,mreff,Mm,Iact=Iact,C=regparams)

In [ ]:
pInv = getInverseParam(Minv,modfun,regfunw,beta,mref,
                       boundsLow,boundsHigh,
                       pcgMaxIter=cgit,maxIter=maxit);

## Run inversion

In [ ]:
m0 = fill(log(halfSpaceCond), size(Iact,2));

In [ ]:
mc,Dc,flag = projGNCG(m0,pInv,pMisRF)

In [ ]:
e

In [ ]:
exportOcTreeModelRoman("inv.con", Minv, (Iact*(e.^mc)) + sigmaBackground);